In [0]:
#conexão com o Google Drive
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

#cria a pasta para manuseio do Google Drive
!mkdir -p drive
!google-drive-ocamlfuse drive

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 131304 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.3-0ubuntu3~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.3-0ubuntu3~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.3-0ubuntu3~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=ht

In [0]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from scipy import stats

import sklearn as sk
import pandas as pd
import numpy as np
import pickle

In [0]:
def process_data(df_train, df_test, df_answer):

	features = list(df_test)
	features.append('NU_NOTA_MT')

	df_answer['NU_INSCRICAO'] = df_test['NU_INSCRICAO']

	df_train = df_train[features]

	# Dropando tabelas que não serão utilizadas

	df_train.drop(['NU_INSCRICAO', 'SG_UF_RESIDENCIA', 'TP_ENSINO', 'TP_DEPENDENCIA_ADM_ESC', 'TP_PRESENCA_CH', 'Q027', 'CO_PROVA_CN', 'CO_PROVA_CH', 'CO_PROVA_LC', 'CO_PROVA_MT'], 1, inplace=True)

	df_test.drop(['NU_INSCRICAO', 'SG_UF_RESIDENCIA', 'TP_ENSINO', 'TP_DEPENDENCIA_ADM_ESC', 'TP_PRESENCA_CH', 'Q027', 'CO_PROVA_CN', 'CO_PROVA_CH', 'CO_PROVA_LC', 'CO_PROVA_MT'], 1, inplace=True)

	# Processado dados categóricos

	df_train['TP_SEXO'].replace('M', 0, inplace=True)
	df_train['TP_SEXO'].replace('F', 1, inplace=True)

	df_test['TP_SEXO'].replace('M', 0, inplace=True)
	df_test['TP_SEXO'].replace('F', 1, inplace=True)

	# Coloquei as notas como zero pq acredito que representam a nota dos alunos que faltaram.

	df_train['NU_NOTA_CN'].fillna(0, inplace=True)
	df_train['NU_NOTA_CH'].fillna(0, inplace=True)
	df_train['NU_NOTA_LC'].fillna(0, inplace=True)
	df_train['NU_NOTA_MT'].fillna(0, inplace=True)
	df_train['NU_NOTA_COMP1'].fillna(0, inplace=True)
	df_train['NU_NOTA_COMP2'].fillna(0, inplace=True)
	df_train['NU_NOTA_COMP3'].fillna(0, inplace=True)
	df_train['NU_NOTA_COMP4'].fillna(0, inplace=True)
	df_train['NU_NOTA_COMP5'].fillna(0, inplace=True)
	df_train['NU_NOTA_REDACAO'].fillna(0, inplace=True)

	df_test['NU_NOTA_CN'].fillna(0, inplace=True)
	df_test['NU_NOTA_CH'].fillna(0, inplace=True)
	df_test['NU_NOTA_LC'].fillna(0, inplace=True)
	df_test['NU_NOTA_COMP1'].fillna(0, inplace=True)
	df_test['NU_NOTA_COMP2'].fillna(0, inplace=True)
	df_test['NU_NOTA_COMP3'].fillna(0, inplace=True)
	df_test['NU_NOTA_COMP4'].fillna(0, inplace=True)
	df_test['NU_NOTA_COMP5'].fillna(0, inplace=True)
	df_test['NU_NOTA_REDACAO'].fillna(0, inplace=True)

	# Coloquei como 8 pq acredito que represente uma situação neutra

	df_train['TP_STATUS_REDACAO'].fillna(8, inplace=True)

	df_test['TP_STATUS_REDACAO'].fillna(8, inplace=True)

	# One_hot_enconding em variáveis categóricas

	df_train = pd.get_dummies(df_train, prefix=['Q001', 'Q002', 'Q006', 'Q024', 'Q025', 'Q026', 'Q047'], columns=['Q001', 'Q002', 'Q006', 'Q024', 'Q025', 'Q026', 'Q047'])

	df_test = pd.get_dummies(df_test, prefix=['Q001', 'Q002', 'Q006', 'Q024', 'Q025', 'Q026', 'Q047'], columns=['Q001', 'Q002', 'Q006', 'Q024', 'Q025', 'Q026', 'Q047'])

	# Utilizando Estatística Descritiva

	notas = ['NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_REDACAO']
	
	df_train['KURTOSIS_NOTAS'] = df_train[notas].kurtosis(axis=1)
	df_train['MEAN_NOTAS'] = df_train[notas].mean(axis=1)
	df_train['MEDIAN_NOTAS'] = df_train[notas].median(axis=1)
	df_train['MAD_NOTAS'] = df_train[notas].mad(axis=1)
	df_train['QUANTILE_NOTAS'] = df_train[notas].quantile(axis=1)
	df_train['SEM_NOTAS'] = df_train[notas].sem(axis=1)
	df_train['SKEW_NOTAS'] = df_train[notas].skew(axis=1)
	df_train['STD_NOTAS'] = df_train[notas].std(axis=1)
	df_train['VAR_NOTAS'] = df_train[notas].var(axis=1)
	df_train['AMP_NOTAS'] = df_train[notas].max(axis=1) - df_train[notas].min(axis=1)
	
	df_train['MEAN_NOTA_CH_LC'] = df_train[['NU_NOTA_CH', 'NU_NOTA_LC']].mean(axis=1)
	df_train['MEAN_NOTA_CH_CN'] = df_train[['NU_NOTA_CH', 'NU_NOTA_CN']].mean(axis=1)
	df_train['MEAN_NOTA_CH_REDACAO'] = df_train[['NU_NOTA_CH', 'NU_NOTA_REDACAO']].mean(axis=1)
	df_train['MEAN_NOTA_CN_LC'] = df_train[['NU_NOTA_CN', 'NU_NOTA_LC']].mean(axis=1)
	df_train['MEAN_NOTA_CN_REDACAO'] = df_train[['NU_NOTA_CN', 'NU_NOTA_REDACAO']].mean(axis=1)
	df_train['MEAN_NOTA_LC_REDACAO'] = df_train[['NU_NOTA_LC', 'NU_NOTA_REDACAO']].mean(axis=1)
	
	notas_red = ['NU_NOTA_COMP1', 'NU_NOTA_COMP2', 'NU_NOTA_COMP3', 'NU_NOTA_COMP4', 'NU_NOTA_COMP5']

	df_train['KURTOSIS_NOTAS_COMP'] = df_train[notas_red].kurtosis(axis=1)
	df_train['MEAN_NOTAS_COMP'] = df_train[notas_red].mean(axis=1)
	df_train['MEDIAN_NOTAS_COMP'] = df_train[notas_red].median(axis=1)
	df_train['MAD_NOTAS_COMP'] = df_train[notas_red].mad(axis=1)
	df_train['QUANTILE_NOTAS_COMP'] = df_train[notas_red].quantile(axis=1)
	df_train['SEM_NOTAS_COMP'] = df_train[notas_red].sem(axis=1)
	df_train['STD_NOTAS_COMP'] = df_train[notas_red].std(axis=1)
	df_train['VAR_NOTAS_COMP'] = df_train[notas_red].var(axis=1)
	df_train['AMP_NOTAS_COMP'] = df_train[notas_red].max(axis=1) - df_train[notas_red].min(axis=1)

	df_train['MEAN_NOTA_COMP1_COMP2'] = df_train[['NU_NOTA_COMP1', 'NU_NOTA_COMP2']].mean(axis=1)
	df_train['MEAN_NOTA_COMP1_COMP3'] = df_train[['NU_NOTA_COMP1', 'NU_NOTA_COMP3']].mean(axis=1)
	df_train['MEAN_NOTA_COMP1_COMP4'] = df_train[['NU_NOTA_COMP1', 'NU_NOTA_COMP4']].mean(axis=1)
	df_train['MEAN_NOTA_COMP1_COMP5'] = df_train[['NU_NOTA_COMP1', 'NU_NOTA_COMP5']].mean(axis=1)
	df_train['MEAN_NOTA_COMP2_COMP3'] = df_train[['NU_NOTA_COMP2', 'NU_NOTA_COMP3']].mean(axis=1)
	df_train['MEAN_NOTA_COMP2_COMP4'] = df_train[['NU_NOTA_COMP2', 'NU_NOTA_COMP4']].mean(axis=1)
	df_train['MEAN_NOTA_COMP2_COMP5'] = df_train[['NU_NOTA_COMP2', 'NU_NOTA_COMP5']].mean(axis=1)
	df_train['MEAN_NOTA_COMP3_COMP4'] = df_train[['NU_NOTA_COMP3', 'NU_NOTA_COMP4']].mean(axis=1)
	df_train['MEAN_NOTA_COMP3_COMP5'] = df_train[['NU_NOTA_COMP3', 'NU_NOTA_COMP5']].mean(axis=1)
	df_train['MEAN_NOTA_COMP4_COMP5'] = df_train[['NU_NOTA_COMP4', 'NU_NOTA_COMP5']].mean(axis=1)
	
	df_test['KURTOSIS_NOTAS'] = df_test[notas].kurtosis(axis=1)
	df_test['MEAN_NOTAS'] = df_test[notas].mean(axis=1)
	df_test['MEDIAN_NOTAS'] = df_test[notas].median(axis=1)
	df_test['MAD_NOTAS'] = df_test[notas].mad(axis=1)
	df_test['QUANTILE_NOTAS'] = df_test[notas].quantile(axis=1)
	df_test['SEM_NOTAS'] = df_test[notas].sem(axis=1)
	df_test['SKEW_NOTAS'] = df_test[notas].skew(axis=1)
	df_test['STD_NOTAS'] = df_test[notas].std(axis=1)
	df_test['VAR_NOTAS'] = df_test[notas].var(axis=1)
	df_test['AMP_NOTAS'] = df_test[notas].max(axis=1) - df_test[notas].min(axis=1)
	
	df_test['MEAN_NOTA_CH_LC'] = df_test[['NU_NOTA_CH', 'NU_NOTA_LC']].mean(axis=1)
	df_test['MEAN_NOTA_CH_CN'] = df_test[['NU_NOTA_CH', 'NU_NOTA_CN']].mean(axis=1)
	df_test['MEAN_NOTA_CH_REDACAO'] = df_test[['NU_NOTA_CH', 'NU_NOTA_REDACAO']].mean(axis=1)
	df_test['MEAN_NOTA_CN_LC'] = df_test[['NU_NOTA_CN', 'NU_NOTA_LC']].mean(axis=1)
	df_test['MEAN_NOTA_CN_REDACAO'] = df_test[['NU_NOTA_CN', 'NU_NOTA_REDACAO']].mean(axis=1)
	df_test['MEAN_NOTA_LC_REDACAO'] = df_test[['NU_NOTA_LC', 'NU_NOTA_REDACAO']].mean(axis=1)
	
	df_test['KURTOSIS_NOTAS_COMP'] = df_test[notas_red].kurtosis(axis=1)
	df_test['MEAN_NOTAS_COMP'] = df_test[notas_red].mean(axis=1)
	df_test['MEDIAN_NOTAS_COMP'] = df_test[notas_red].median(axis=1)
	df_test['MAD_NOTAS_COMP'] = df_test[notas_red].mad(axis=1)
	df_test['QUANTILE_NOTAS_COMP'] = df_test[notas_red].quantile(axis=1)
	df_test['SEM_NOTAS_COMP'] = df_test[notas_red].sem(axis=1)
	df_test['STD_NOTAS_COMP'] = df_test[notas_red].std(axis=1)
	df_test['VAR_NOTAS_COMP'] = df_test[notas_red].var(axis=1)
	df_test['AMP_NOTAS_COMP'] = df_test[notas_red].max(axis=1) - df_test[notas_red].min(axis=1)

	df_test['MEAN_NOTA_COMP1_COMP2'] = df_test[['NU_NOTA_COMP1', 'NU_NOTA_COMP2']].mean(axis=1)
	df_test['MEAN_NOTA_COMP1_COMP3'] = df_test[['NU_NOTA_COMP1', 'NU_NOTA_COMP3']].mean(axis=1)
	df_test['MEAN_NOTA_COMP1_COMP4'] = df_test[['NU_NOTA_COMP1', 'NU_NOTA_COMP4']].mean(axis=1)
	df_test['MEAN_NOTA_COMP1_COMP5'] = df_test[['NU_NOTA_COMP1', 'NU_NOTA_COMP5']].mean(axis=1)
	df_test['MEAN_NOTA_COMP2_COMP3'] = df_test[['NU_NOTA_COMP2', 'NU_NOTA_COMP3']].mean(axis=1)
	df_test['MEAN_NOTA_COMP2_COMP4'] = df_test[['NU_NOTA_COMP2', 'NU_NOTA_COMP4']].mean(axis=1)
	df_test['MEAN_NOTA_COMP2_COMP5'] = df_test[['NU_NOTA_COMP2', 'NU_NOTA_COMP5']].mean(axis=1)
	df_test['MEAN_NOTA_COMP3_COMP4'] = df_test[['NU_NOTA_COMP3', 'NU_NOTA_COMP4']].mean(axis=1)
	df_test['MEAN_NOTA_COMP3_COMP5'] = df_test[['NU_NOTA_COMP3', 'NU_NOTA_COMP5']].mean(axis=1)
	df_test['MEAN_NOTA_COMP4_COMP5'] = df_test[['NU_NOTA_COMP4', 'NU_NOTA_COMP5']].mean(axis=1)

	return df_train, df_test, df_answer

In [0]:
def convert_to_zero(filename):

	data = pd.read_csv(filename+'.csv')

	for i in range(0, data.shape[0]):
		if data['NU_NOTA_MT'][i] < 80:
			data['NU_NOTA_MT'][i] = 0.00

	data.to_csv('/content/drive/ML/data-codenation-challenge/answer_with_stats_zscore8.csv', index=False, header=True)

	return data

In [0]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression

def ft_importance(k=50):

	df_train = pd.read_csv('/content/drive/ML/data-codenation-challenge/train.csv')
	df_test = pd.read_csv('/content/drive/ML/data-codenation-challenge/test.csv')
	df_answer = pd.DataFrame()

	df_train, df_test, df_answer = process_data(df_train, df_test, df_answer)

	label = df_train['NU_NOTA_MT']

	df_train.drop(['NU_NOTA_MT'], axis=1, inplace=True)

	# Apply SelectKBest class to extract top 10 best features

	bestfeatures = SelectKBest(score_func=f_regression, k=k)

	fit = bestfeatures.fit(df_train,label)
	
	dfscores = pd.DataFrame(fit.scores_)
	
	dfcolumns = pd.DataFrame(df_train.columns)

	# Concat two dataframes for better visualization 

	featureScores = pd.concat([dfcolumns,dfscores],axis=1)

	# Naming the dataframe columns

	featureScores.columns = ['Specs','Score']

	return featureScores.nlargest(k,'Score')

In [0]:
def random_search(df_train, df_test, df_answer, label):

	# Number of trees in random forest
	n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
	# Number of features to consider at every split
	max_features = ['auto', 'sqrt']
	# Maximum number of levels in tree
	max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
	max_depth.append(None)
	# Minimum number of samples required to split a node
	min_samples_split = [2, 5, 10]
	# Minimum number of samples required at each leaf node
	min_samples_leaf = [1, 2, 4]
	# Method of selecting samples for training each tree
	bootstrap = [True, False]
	# Create the random grid
	random_grid = {'n_estimators': n_estimators,
	               'max_features': max_features,
	               'max_depth': max_depth,
	               'min_samples_split': min_samples_split,
	               'min_samples_leaf': min_samples_leaf,
	               'bootstrap': bootstrap}
	print(random_grid)

	# Use the random grid to search for best hyperparameters
	# First create the base model to tune
	rf = RandomForestRegressor()
	# Random search of parameters, using 3 fold cross validation, 
	# search across 100 different combinations, and use all available cores
	rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 50, cv = 3, verbose=3, random_state=42, n_jobs = -1)

	# Fit the random search model
	rf_random.fit(df_train, label)

	bestRF = rf_random.best_estimator_
	
	print(bestRF)
	
	predictions = bestRF.predict(df_test)

	df_answer['NU_NOTA_MT'] = np.around(predictions,2)

	df_answer.to_csv('/content/drive/ML/data-codenation-challenge/regressor_answer.csv', index=False, header=True)

In [0]:
df_train = pd.read_csv('/content/drive/ML/data-codenation-challenge/train.csv')
df_test = pd.read_csv('/content/drive/ML/data-codenation-challenge/test.csv')
df_answer = pd.DataFrame()

df_train, df_test, df_answer = process_data(df_train, df_test, df_answer)

z = np.abs(stats.zscore(df_train))

nan_values_index = np.isnan(z)

z[nan_values_index] = 0

df_train = df_train[(z < 8).all(axis=1)]

print(df_train.shape)

label = df_train['NU_NOTA_MT']

df_train.drop(['NU_NOTA_MT'], axis=1, inplace=True)

random_search(df_train, df_test, df_answer, label)

data = convert_to_zero('/content/drive/ML/data-codenation-challenge/regressor_answer')

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)
/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:6586: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:6130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


(12660, 114)
{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]}
Fitting 3 folds for each of 50 candidates, totalling 150 fits


/usr/local/lib/python3.6/dist-packages/scipy/stats/stats.py:2281: RuntimeWarning: invalid value encountered in true_divide
  return (a - mns) / sstd
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed: 45.6min
[Parallel(n_jobs=-1)]: Done 124 tasks      | elapsed: 190.9min
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed: 212.6min finished


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=10,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=4, min_samples_split=5,
           min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
